# HMM to estimate Funnel position

## Import libraries and constants

In [ ]:
# Constants
from config.CONSTANTS_HMM import *
from config.execution_parameters import *

# Project libraries
import src.hmm_package.generate_hmm
from src.hmm_package.generate_hmm import *
from src.plot_and_print_info.plots_and_print_info import *

# Built in libraries
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd
import importlib

In [ ]:
importlib.reload(src.hmm_package.generate_hmm)
from src.hmm_package.generate_hmm import *

## Compute Observation and Adstock

In [ ]:
# Generate Test observation
observation = simulate_observations()

# Compute Adstock
adstock = compute_adstock(observation=observation)

In [ ]:
adstock

## Real HMM to Estimate

In [ ]:
# Generate the distributions to build the Real HMM
#NB The variable STATES_ARE_OBSERVABLE is set in config/execution_parameters.
hmm_distributions = generate_hmm_distributions(states_observable=True, adstock=adstock)

# Create Real HMM to fit
real_hmm = tfd.HiddenMarkovModel(
    initial_distribution=hmm_distributions['initial_distribution'],
    transition_distribution=hmm_distributions['transition_distribution'],
    observation_distribution=hmm_distributions['observation_distribution'],
    time_varying_transition_distribution=True,
    num_steps=time+1
)

# Sample emissions
# TODO: once everything works, move this function in a test environment
emission_real = real_hmm.sample(1).numpy()[0]

### Visualize the Real HMM

In [ ]:
# Count conversions from sampled data
tot_conversions = count_conversions(emission_real, STATES_ARE_OBSERVABLE)

plot_sample_emissions(real_hmm)

## Build model to fit

In [ ]:
# STATES_ARE_OBSERVABLE is defined in CONSTANTS_HMM
model = build_hmm_to_fit( states_observable=STATES_ARE_OBSERVABLE )

compiler = CompilerInfo(LR_EXPONENTIAL_DECAY)
model.compile(
    loss = compiler.loss,
    optimizer = compiler.optimizer,
    run_eagerly = True
)

## Fit the model

In [ ]:
model_fitted = model.fit(adstock,
                         emission_real,
                         epochs=EPOCHS,
                         batch_size=BATCH_SIZE,
                         verbose=True)

In [ ]:
fit_model(model, adstock, emission_real)